# Đồ án cuối kỳ
 Thành viên:
 
 Họ tên: Trần Phúc Nguyên
 MSSV: 18120488
 
 Họ tên: Trần Thành Long
 MSSV: 18120455

## Import các thư viện cần thiết

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# YOUR CODE HERE (OPTION) 

---

## Thu thập dữ liệu

Trong bài này, ta sẽ dùng dữ liệu đã được thu thập sẵn là file "vgsales.csv" đính kèm; đây là dữ liệu về danh sách các video game đã bán hơn 100,000 bản("vgsales" là viết tắt của "video games sales"). Dữ liệu này được lấy từ trang Kaggle link trang:(https://www.kaggle.com/gregorut/videogamesales). Lần cuối dữ liệu được update là 26/10/2016.
Dữ liệu được thu thập bằng cách sử dụng BeautifulSoup của Python(link script: https://github.com/GregorUT/vgchartzScrape.) để cào dữ liệu từ website vgchartz.com.
Dữ liệu không có License.

---

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [2]:
vgsales_df = pd.read_csv('vgsales.csv')
vgsales_df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột? 

In [3]:
a=vgsales_df.shape
n_row=a[0]
n_col=a[1]
print(n_row)
print(n_col)

16598
11


Vậy dữ liệu có tất cả **16598 dòng** và **11 cột**

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Theo mô tả của trang Kaggle và quan sát sơ bộ về dữ liệu thì một dòng cho biết thông tin về video game (đã bán hơn 100,000 bản) cũng như doanh số bán ra của video game này theo từng khu vực. Có vẻ không có dòng nào "lạc loài".

### Dữ liệu có các dòng bị lặp không? 

In [4]:
have_duplicated_rows=False
checkdup=vgsales_df.duplicated()
if checkdup.any() == True:
    have_duplicated_rows=True
print(have_duplicated_rows)

False


**Như vậy dữ liệu không có dòng nào bị lặp.**

### Mỗi cột có ý nghĩa gì?

Dưới đây là mô tả của trang kaggle "Video Game Sales" về các cột trong file vgsales.csv:

-   **Rank** : Thứ hạng dựa trên tổng bán ra
-   **Name** : Tên video game
-  **Platform** : Nền tảng của video game (i.e. PC,PS4, etc.)
-   **Year** : Năm video game được bán ra
-   **Genre** : Thể loại game
-   **Publisher** : Nhà phát hành video game
-   **NA_Sales** : Doanh thu tại North America (in millions)
-   **EU_Sales** : Doanh thu tại Europe (in millions)
-   **JP_Sales** : Doanh thu tại Japan (in millions)
-   **Other_Sales** : Doanh thu tại các khu vực còn lại (in millions)
-   **Global_Sales** : Tổng doanh thu toàn cầu.


### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không? 

In [5]:
col_dtypes=pd.Series(vgsales_df.dtypes,index=vgsales_df.columns)
print(col_dtypes)

Rank              int64
Name             object
Platform         object
Year            float64
Genre            object
Publisher        object
NA_Sales        float64
EU_Sales        float64
JP_Sales        float64
Other_Sales     float64
Global_Sales    float64
dtype: object


Quan sát từ dòng code trên ta thấy được có 4 cột mang kiểu dữ liệu **object** đó là: "Name","Platform","Genre","Publisher". Để biết rõ hơn về kiểu dữ liệu của các cột này ta làm như sau:

- Đầu tiên ta xây dựng một hàm xử lý việc trả ra dữ liệu của cột:

In [6]:
def open_object_dtype(s):
    dtypes = set()
    i=0
    dtypes.add(type(s.iloc[i]))
    i=i+1
    while i<len(s):
        a=type(s.iloc[i])
        if (a in dtypes)== False:
            dtypes.add(a)
        i=i+1
    
    
    return dtypes


- Từ hàm xử lý kiểu dữ liệu đã xây dựng ta tiến hành kiểm tra dữ liệu thật của từng cột như sau:

In [7]:
print('Dữ liệu cột Name',open_object_dtype(vgsales_df['Name']))
print('Dữ liệu cột Platform',open_object_dtype(vgsales_df['Platform']))
print('Dữ liệu cột Genre',open_object_dtype(vgsales_df['Genre']))
print('Dữ liệu cột Publisher',open_object_dtype(vgsales_df['Publisher']))

Dữ liệu cột Name {<class 'str'>}
Dữ liệu cột Platform {<class 'str'>}
Dữ liệu cột Genre {<class 'str'>}
Dữ liệu cột Publisher {<class 'str'>, <class 'float'>}


**Vấn đề cần tiền xử lý **
- Cột "Publisher" có dtype gồm 2 loại là str và float. Nếu kiểm tra thêm thì chúng ta sẽ thấy float là ứng với NaN, còn str là ứng với tên của nhà phát hành. Để có thể tiếp tục khám phá thêm về cột này, ta cũng sẽ thực hiện bước tiền xử lý là chuyển tất cả sang dtype là str. Những cột có giá trị NaN sẽ được để khoảng trống.

---

## Tiền xử lý

### Chuyển dtype của cột Publisher sang kiểu str

Tiến hành thay thế các giá trị NaN bằng khoảng ' ',các giá trị còn lại chuyển thành kiểu str 

In [30]:
vgsales_df['Publisher'] = vgsales_df['Publisher'].fillna('').astype('str')
open_object_dtype(vgsales_df['Publisher'])

{str}

---

## Khám phá dữ liệu (tiếp tục)

### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

Hiện có 5 cột có vẻ là thuộc nhóm numeric: "NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales".  

Với mỗi cột numeric sẽ tiến hành tính tỉ lệ % giá trị thiếu (từ 0 đến 100), min, max.Và lưu kết quả vào dataframe `nume_col_profiles_df`. Dataframe này có 3 dòng là "missing_ratio", "min", "max"; và có 5 cột là "NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales".  

In [9]:
a_df=vgsales_df.loc[:,['Rank','Year', "NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"]]
def missing_ratio(s):
    return s.isna().mean() * 100
nume_col_profiles_df=a_df.agg([missing_ratio, pd.Series.min, pd.Series.max])
nume_col_profiles_df

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
missing_ratio,0.0,1.632727,0.00,0.00,0.00,0.00,0.00
min,1.0,1980.000000,0.00,0.00,0.00,0.00,0.01
max,16600.0,2020.000000,41.49,29.02,10.22,10.57,82.74


Không có giá trị thiếu và các giá trị max min không gì "bất thường"

### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

Hiện có 6 cột có vẻ là thuộc nhóm categoric: "Rank", "Name", "Year", "Publisher", "Platform","Gerne".  

Với mỗi cột categorical, chúng ta sẽ tính tỉ lệ % giá trị thiếu (từ 0 đến 100), số lượng giá trị khác nhau, list/array một vài các giá trị khác nhau. Dataframe kết quả có 3 dòng là "missing_ratio", "num_diff_vals", "some_diff_vals"; và có 6 cột là "Rank", "Name", "Year", "Publisher", "Platform","Gerne".


In [31]:
cate_col_profiles_df = vgsales_df[['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher']].agg(\
    [missing_ratio, 'nunique', 'unique'])
cate_col_profiles_df = cate_col_profiles_df.rename(index={'nunique':'num_diff_vals', 'unique':'some_diff_vals'})
cate_col_profiles_df

,Rank,Name,Platform,Year,Genre,Publisher
missing_ratio,0,0,0,1.63273,0,0
num_diff_vals,16598,11493,31,39,12,579
some_diff_vals,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[Wii Sports, Super Mario Bros., Mario Kart Wii...","[Wii, NES, GB, DS, X360, PS3, PS2, SNES, GBA, ...","[2006.0, 1985.0, 2008.0, 2009.0, 1996.0, 1989....","[Sports, Platform, Racing, Role-Playing, Puzzl...","[Nintendo, Microsoft Game Studios, Take-Two In..."
